<a href="https://colab.research.google.com/github/kateburovova/Communication_Patterns_Investigation/blob/mainbranch/finetune_RuBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df_labels = pd.read_csv('/content/final_labels.csv', index_col=[0])

In [ ]:
df_mention = df_labels[['text', 'Mention']]
df_emotion = df_labels[['text', 'Emotion']]

In [ ]:
mention_label_mapping = {'так': 1, 'ні': 0, 'не можу визначитись із відповіддю': -1}
df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)

<ipython-input-4-1554e77b6fff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)


In [ ]:
# dropping lines where labelers are unsure
df_mention = df_mention[df_mention['labels']!=-1]
len(df_mention)

3518

In [ ]:
emotion_label_mapping = {'так, присутня негативна': 1,
                         'ні, оцінка не присутня': 0,
                         'не можу визначитись з правильною відповіддю': -1,
                         'так, присутня позитивна': 2}
df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)

<ipython-input-6-b63af5414c05>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)


In [ ]:
df_emotion = df_emotion[df_emotion.labels!=-1] #dropping those regarding which labelers are unsure

In [ ]:
len(df_emotion)


3495

In [ ]:
pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
import random
from transformers import set_seed
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
def set_random_seeds(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    set_seed(seed)  

seed = 42
set_random_seeds(seed)

# Mention

In [ ]:
train_data, val_data = train_test_split(df_mention, test_size=0.2, random_state=42)
max_length = 512

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")
train_encodings = tokenizer(train_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)
val_encodings = tokenizer(val_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)

In [ ]:
class BinaryClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BinaryClassificationDataset(train_encodings, train_data["labels"].tolist())
val_dataset = BinaryClassificationDataset(val_encodings, val_data["labels"].tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
# Load the pre-trained model
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
)

def compute_accuracy(predictions, labels):
    preds = predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     return compute_accuracy(predictions, labels)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(project="binary_sentence_classification", name="rubert_mention_finetuning_colab")


wandb: Currently logged in as: kate-burovova (brvva_team). Use `wandb login --relogin` to force relogin


In [ ]:
# Start training
best_model_TrainOutput = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,0.078500,0.359001,0.949725
2,0.000300,0.320047,0.958571


In [ ]:
best_model_TrainOutput

TrainOutput(global_step=704, training_loss=0.03006206781190651, metrics={'train_runtime': 150.8492, 'train_samples_per_second': 37.309, 'train_steps_per_second': 4.667, 'total_flos': 1304366890281840.0, 'train_loss': 0.03006206781190651, 'epoch': 2.0})

In [ ]:
wandb.finish()

eval/f1_score,▄█▅▄▆▇▆▃▅▆▆▆█▇▇▇▆▇▇▇▆▇▇▆▁▁▆▆▆▆
eval/loss,▁▁▃▄▂▃▃▄▄▄▃▃▂▃▃▃▃▃▄▄▄▄▄▄██▅▅▅▅
eval/runtime,▄▅▆▃▄▅▄▃▅▄▄▂▁▁▁▂▂▁▃▂▁▁▄▃▃▄█▃▂▂
eval/samples_per_second,▅▄▃▆▅▄▅▆▄▅▅▇███▇▇█▆▇██▅▆▆▅▁▆▇▇
eval/steps_per_second,▅▄▃▆▅▄▅▆▄▅▅▇███▇▇█▆▇██▅▆▆▅▁▆▇▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▂▄█▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
trained_model = trainer.model

In [ ]:
output_directory = "./saved_model"
trained_model.save_pretrained(output_directory)

#Emotion

In [ ]:
train_data, val_data = train_test_split(df_emotion, test_size=0.2, random_state=42)
max_length = 512

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
train_encodings = tokenizer(train_data["text"].tolist(), padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_data["text"].tolist(), padding=True, truncation=True, max_length=512)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_data['labels'].tolist())
val_dataset = CustomDataset(val_encodings, val_data['labels'].tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=3)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:
wandb.init(project="binary_sentence_classification", name="1_epoch_rubert_emotion_finetuning_colab")

train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▅▅▆▃▆▄▃▃▂▅▅▂▆▆▄▃▄▂▁▄▂▇▂▁▄▁▅▃▃▂▅▂▃▂
train/epoch,1.0
train/global_step,350
train/learning_rate,0.0
train/loss,0.2623


In [ ]:
training_output = trainer.train()

In [ ]:
wandb.finish()

train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▇▆▇▅▄▃▄▄▂▆▃▂▆▅▃▄▄▃▁▅▃▅▃▂▄▁▅▂▄▃▃▂▂▃
train/epoch,1.0
train/global_step,350
train/learning_rate,0.0
train/loss,0.3398


We could either stop at 1 epoch or try more regularization apart from the weitghs decay that we already use.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=3, hidden_dropout_prob=0.5)
config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", config=config)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
from datetime import datetime

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"run_{current_time}"

In [ ]:
from sklearn.metrics import f1_score

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
    run_name = run_name
)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:
wandb.init(project="binary_sentence_classification", name="rubert_emotion_finetuning_colab_no_dropout_1ep")

train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▆▆▅▆▅▃▄▄▃▁▇▆▄██▄▃▅▃▁▇▅▇▄▂▃▂▇▅█▅▇▅▅▆
train/epoch,1.0
train/global_step,350
train/learning_rate,0.0
train/loss,0.37


In [ ]:
training_output = trainer.train()
wandb.finish()

Epoch,Training Loss,Validation Loss,F1 Score
1,0.489200,0.449620,0.896527


eval/f1_score,▁
eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▄▆▂▄▁▂▂▆▂▁█▅▂▆▆▃▃▃▃▂▄▄▄▃▁▂▂▆▅▇▃▇▄▅█
train/total_flos,▁
train/train_loss,▁


In [ ]:
trained_model = trainer.model

NameError: ignored

In [ ]:
output_directory = "./saved_2model"
trained_model.save_pretrained(output_directory)

In [ ]:
trained_model = trained_model.to('cuda:0')

In [ ]:
input_text = 'Тут понимаешь всё про метафизику этой войны. «Укропы» под покровом Запада вышли не против России — они вышли на войну с Богом.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

In [ ]:
with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(1, device='cuda:0')

In [ ]:
input_text = 'Ну и мудила украинец конечно.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(1, device='cuda:0')

In [ ]:
input_text = 'Красавчик украинец, супер.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(0, device='cuda:0')

In [ ]:
input_text = 'Ну и мудила конечно.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(0, device='cuda:0')

In [ ]:
input_text = 'Какие хорошенькие котики, прелесть'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(0, device='cuda:0')

## Emotion from finetuned

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
set_seed(seed)


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [ ]:
text1 = 'какая милая кошечка'
text2 = 'какая мразь конечно, ну мудила!'
text3 = 'нейтральное предложение'

In [ ]:
predict(text1)

array([1])

In [ ]:
predict(text2)

array([2])

In [ ]:
predict(text3)

array([0])

In [ ]:
df_emotion_remapped = df_emotion.copy()


In [ ]:
df_emotion_remapped['labels'] = df_emotion_remapped['labels'].replace({0: 0, 
                                                              1: 2,
                                                              2: 1})


In [ ]:
train_data, val_data = train_test_split(df_emotion_remapped, test_size=0.2, random_state=seed)
max_length = 512

In [ ]:
def preprocess_data(data):
    encodings = tokenizer(
        data['text'].tolist(),
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors='pt'
    )
    labels = torch.tensor(data['labels'].tolist())
    return encodings, labels

train_encodings, train_labels = preprocess_data(train_data)
val_encodings, val_labels = preprocess_data(val_data)

In [ ]:
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
from sklearn.metrics import f1_score

from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=1,  # Number of evaluations with no improvement after which training will be stopped
    early_stopping_threshold=0.0,  # Minimum relative improvement in the monitored metric required to qualify as an improvement
)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    seed=seed,
    metric_for_best_model="f1_score",
    greater_is_better=True,
    learning_rate=2e-5,
    report_to="wandb"
)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)


In [ ]:
wandb.init(project="binary_sentence_classification", name="rubert_pretrained_em_ft_colab")

wandb: Currently logged in as: kate-burovova (brvva_team). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()
trainer.evaluate()
wandb.finish()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-25-9619334f4396>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-25-9619334f4396>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss,F1 Score
1,0.592900,0.683727,0.626597
2,0.475100,0.537509,0.777585
3,0.505700,0.526947,0.797182
4,0.359000,0.469140,0.818214
5,0.526100,0.460688,0.812196


<ipython-input-25-9619334f4396>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-25-9619334f4396>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-25-9619334f4396>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-25-9619334f4396>:10: UserWarning: To copy construct from a tensor, it is recommended 

eval/f1_score,▁▇▇███
eval/loss,█▃▃▁▁▁
eval/runtime,█▇▇█▆▁
eval/samples_per_second,▁▂▂▁▃█
eval/steps_per_second,▁▂▂▁▃█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▆▄▆▆▄▄▄▅▃▄▃▄▃▃▂▄▃▄▂▃▂▃▂▂▄▂▂▂▂▂▂▂▁▂▂▂▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
# from transformers import AutoModelForSequenceClassification

# best_model_path = training_args.output_dir + "/best_model"
# best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path, return_dict=True)

In [ ]:
trained_model = trainer.model

In [ ]:
output_directory = "./saved_2model"
trained_model.save_pretrained(output_directory)

In [ ]:
trained_model = trained_model.to('cuda:0')
input_text = 'Тут понимаешь всё про метафизику этой войны. «Укропы» под покровом Запада вышли не против России — они вышли на войну с Богом.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

In [ ]:
with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]


tensor(2, device='cuda:0')

In [ ]:
input_text = 'Плохой украинец.'
input_tokens = tokenizer(input_text, padding=True, max_length=512, truncation=True, return_tensors="pt")
input_tokens = {k: v.to('cuda:0') for k, v in input_tokens.items()}

with torch.no_grad():
    logits = trained_model(**input_tokens).logits

predictions = torch.argmax(logits, dim=-1)
predictions[0]

tensor(2, device='cuda:0')

In [ ]:
df_emotion_remapped

,text,Emotion,labels
1219,"Всвязи с этим немного поправлю коллег ⤵️ ""Они...","так, присутня негативна",2
1218,Литературный критик Галина Юзефович о новом ро...,"ні, оцінка не присутня",0
1591,Почему на базах неонацистов стоят языческие ис...,"так, присутня негативна",2
1198,Группа добровольцев-медиков из Чеченской Респу...,"так, присутня негативна",2
3247,"ВСУшники, переходите на сторону добра, у нас т...","ні, оцінка не присутня",0
...,...,...,...
3613,Утренний брифинг Минобороны России: ▪️ россий...,"ні, оцінка не присутня",0
3612,И понеслась мазепинщино-петлюровщино-бандеровщ...,"так, присутня негативна",2
4121,Наш соратник по русскому движению Алексей Сели...,"так, присутня негативна",2
4120,Хорошее видео от 4 бригады НМ ЛНР https://t.me...,"ні, оцінка не присутня",0
